<a href="https://colab.research.google.com/github/aisaza609/Datos_a_la_U_grupo_2/blob/main/Normalizacion_Dataset_Neiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import io
from io import BytesIO
import random
import ipywidgets as widgets
from IPython.display import display
!pip install gdown
import gdown
from datetime import datetime
from datetime import timedelta
%matplotlib inline

# Dataset Hospital Neiva nivel 3

Para simular el hospital de nivel 3, se realizo con los datos del Hospital Universitario Hernando Moncaleano Perdomo, Neiva [Dataset de Neiva](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Perfil-de-morbilidad/5c4n-qdbv/about_data)

In [21]:
# Definimos el ID del archivo de Google Drive y generamos la URL de descarga
file_id = '1pHF94shp0VZPkBXD4QKZM1XnLq31COL5'  # Reemplaza esto con el ID del archivo en Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Descargamos el archivo y lo guardamos con el nombre df_neiva.csv
output = 'df_neiva.csv'
gdown.download(url, output, quiet=False)

# Cargamos el archivo en un DataFrame de pandas llamado df_neiva para empezar a trabajar con los datos
df_neiva = pd.read_csv('df_neiva.csv')
print(df_neiva.head())  # Verificamos las primeras filas para asegurarnos de que se cargó correctamente

Downloading...
From: https://drive.google.com/uc?id=1pHF94shp0VZPkBXD4QKZM1XnLq31COL5
To: /content/df_neiva.csv
100%|██████████| 4.24M/4.24M [00:00<00:00, 63.4MB/s]


  TIPO        FECHA_NACIMIENTO  MUNICIPIO SEXO COD_DX  \
0   CC  09/24/1972 12:00:00 AM     ITAGUI    F   D172   
1   CC  01/29/1981 12:00:00 AM  SAN PEDRO    M   S523   
2   CC  09/11/1973 12:00:00 AM     ITAGUI    F   S324   
3   RC  04/07/2018 12:00:00 AM   ENVIGADO    F   J042   
4   CC  01/03/1994 12:00:00 AM   MEDELLIN    F   S823   

                                           NOMBRE_DX  \
0  TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...   
1                  FRACTURA DE LA DIAFISIS DEL RADIO   
2                             FRACTURA DEL ACETABULO   
3                            LARINGOTRAQUEITIS AGUDA   
4       FRACTURA DE LA EPIFISIS INFERIOR DE LA TIBIA   

                                            ENTIDAD  
0                                      PARTICULARES  
1                    SEGUROS GENERALES SURAMERICANA  
2  ADRES (ADMINISTRADORA DE LOS RECURSOS DEL SGSSS)  
3                        EPS SURA CONTRIBUTIVO 2020  
4                    ALIANZA MEDELLIN ANTIOQUIA SA

## Fecha de ingreso

El dataset no cuenta con fechas de ingreso como tal, por eso generamos aleatoriamente desde el 2021 hasta la fecha de hoy

In [22]:
# Definir el número de filas que necesito en la columna de fechas aleatorias
num_filas = len(df_neiva)

# Función para generar fechas aleatorias a partir del año 2021 en adelante
def generar_fecha_aleatoria():
    # Establezco la fecha de inicio como 1 de enero de 2021
    fecha_inicio = datetime(2021, 1, 1)
    # Defino un rango de días (por ejemplo, 3 años = 3*365 días)
    rango_dias = 3 * 365
    # Genero un número de días aleatorio dentro del rango para sumar a la fecha de inicio
    fecha_aleatoria = fecha_inicio + timedelta(days=np.random.randint(0, rango_dias))
    return fecha_aleatoria

# Creo la columna 'FechaIngreso' en el dataframe con fechas aleatorias
df_neiva['FechaIngreso'] = [generar_fecha_aleatoria() for _ in range(num_filas)]

# Verifico el resultado
print(df_neiva[['FechaIngreso']].head())


  FechaIngreso
0   2023-12-04
1   2022-09-11
2   2022-05-06
3   2023-05-11
4   2021-09-08


## Nivel del hospital

El nivel del hospital se consulto en el dataset de REPS

In [23]:
# Agregar una nueva columna 'NivelHospital' con valor 'Nivel 3' en todo el dataset
df_neiva['NivelHospital'] = '3'

Para normalizar primeo cambiamos el nombre a las columnas ya existentes para que coincidan

In [24]:
df_neiva.rename(columns={'COD_DX': 'CodigoDiagnostico'}, inplace=True)
df_neiva.rename(columns={'NOMBRE_DX': 'NombreDiagnostico'}, inplace=True)
df_neiva.rename(columns={'TIPO': 'tipo de identificación'}, inplace=True)
df_neiva.rename(columns={'SEXO': 'Sexo'}, inplace=True)
df_neiva.rename(columns={'ENTIDAD': 'Entidad'}, inplace=True)



Ahora vamos añadiendo las columnas que no se encuentan en el dataset

## LA CEDULA

In [25]:
# Generar una lista de números únicos de 10 dígitos con la misma cantidad de filas que el DataFrame
n_documentos = random.sample(range(1000000000, 9999999999), len(df_neiva))

# Asignar la lista generada a la nueva columna 'NDocumento'
df_neiva['NDocumento'] = n_documentos

# Muestra las primeras filas para verificar
print(df_neiva[['NDocumento']].head())

   NDocumento
0  6362268471
1  1615029526
2  5399996004
3  9028510898
4  7359888768


## ingreso

In [26]:
# Generar una lista de números únicos de 7 dígitos con la misma cantidad de filas que el DataFrame para el código de ingreso
n_ingreso = random.sample(range(1000000, 9999999), len(df_neiva))

# Asignar la lista generada a la nueva columna 'Ingreso'
df_neiva['Ingreso'] = n_ingreso

# Muestra las primeras filas para verificar
print(df_neiva[['Ingreso']].head())

   Ingreso
0  5665894
1  7631352
2  6033245
3  6240010
4  2356060


## Edad

El dataset no tiene edad pero tiene fecha de nacimiento, procedemos a calcularla

In [27]:


# Convertir la columna de fecha de nacimiento a formato datetime
df_neiva['FECHA_NACIMIENTO'] = pd.to_datetime(df_neiva['FECHA_NACIMIENTO'], errors='coerce')

# Calcular la edad basándonos en la fecha actual y la fecha de nacimiento
df_neiva['Edad'] = df_neiva['FECHA_NACIMIENTO'].apply(lambda x: (datetime.now() - x).days // 365 if pd.notnull(x) else None)

# Mostrar algunas filas para verificar que la columna Edad se haya creado correctamente
print(df_neiva[['FECHA_NACIMIENTO', 'Edad']].head())


  FECHA_NACIMIENTO  Edad
0       1972-09-24    52
1       1981-01-29    43
2       1973-09-11    51
3       2018-04-07     6
4       1994-01-03    30


Eliminamos la columna de fecha de nacimiento ya que calculada la edad, ya no se necesita dicha columna. Tambien eliminamos Municipio ya que no todos los datasets cuentan con este atributo                                             

In [28]:
df_neiva.drop('FECHA_NACIMIENTO', axis=1, inplace=True)
df_neiva.drop('MUNICIPIO', axis=1, inplace=True)


## Dataset CE-10

Ahora para añadir la descripcion del codigo diagnostico cargamos el dataset de CE-10 de sispro.

In [29]:
# URL directa al archivo en GitHub
url3 = 'https://raw.githubusercontent.com/aisaza609/Datos_a_la_U_grupo_2/main/Datasets%20externos/TablaReferencia_CIE10__1.xlsx'

# Descargar el archivo desde GitHub
response = requests.get(url3)
if response.status_code == 200:
    # Leer el contenido con pandas, especificando el motor como 'openpyxl'
    excel_data = pd.read_excel(BytesIO(response.content), engine='openpyxl')
    # Mostrar las primeras filas para verificar

else:
    print("No se pudo descargar el archivo.")
excel_data.head()

,Tabla,Codigo,Nombre,Descripcion,Habilitado,Aplicacion,IsStandardGEL,IsStandardMSPS,Extra_I:AplicaASexo,Extra_II:EdadMinima,...,Extra_V,Extra_VI:Capitulo,Extra_VII:Grupo,Extra_VIII:SubGrupo,Extra_IX:Categoria,Extra_X:Sexo,ValorRegistro,UsuarioResponsable,Fecha_Actualizacion,IsPublicPrivate
0,CIE10,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,0.0,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
1,CIE10,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
2,CIE10,A009,"COLERA, NO ESPECIFICADO",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
3,CIE10,A010,FIEBRE TIFOIDEA,FIEBRES TIFOIDEA Y PARATIFOIDEA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A01,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
4,CIE10,A011,FIEBRE PARATIFOIDEA A,FIEBRES TIFOIDEA Y PARATIFOIDEA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A01,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN


Como del dataset de codigos solo necesitamos el nombres, codigo y descripcion eliminamos el resto de columnas

In [30]:
excel_data = excel_data.iloc[:, 1:4]

# Mostrar el resultado para verificar
excel_data.head()

,Codigo,Nombre,Descripcion
0,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",COLERA
1,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",COLERA
2,A009,"COLERA, NO ESPECIFICADO",COLERA
3,A010,FIEBRE TIFOIDEA,FIEBRES TIFOIDEA Y PARATIFOIDEA
4,A011,FIEBRE PARATIFOIDEA A,FIEBRES TIFOIDEA Y PARATIFOIDEA


## Descripcion de Diagnostico

El dataset no cuenta la columna de descripcion de codigo, realizamos una comparacion con el codigo y rellenamos esa informacion la nueva columna

In [31]:
# Realizar un merge entre df_neiva y excel_data en base a 'CodigoDiagnostico' de df_neiva y 'Codigo' de excel_data,
# renombrando la columna 'Descripcion' como 'DxSindromatico' directamente en el merge
df_neiva = df_neiva.merge(excel_data[['Codigo', 'Descripcion']].rename(columns={'Descripcion': 'DxSindromatico'}),
                          left_on='CodigoDiagnostico', right_on='Codigo', how='left')

# Eliminar la columna 'Codigo' adicional generada tras el merge
df_neiva = df_neiva.drop(columns=['Codigo'])

# Verificar el resultado para asegurar que 'DxSindromatico' se ha agregado correctamente
print(df_neiva[['CodigoDiagnostico', 'DxSindromatico']].head())


  CodigoDiagnostico                                DxSindromatico
0              D172                  TUMORES BENIGNOS LIPOMATOSOS
1              S523                        FRACTURA DEL ANTEBRAZO
2              S324  FRACTURA DE LA COLUMNA LUMBAR Y DE LA PELVIS
3              J042                LARINGITIS Y TRAQUEITIS AGUDAS
4              S823         FRACTURA PIERNA, INCLUSIVE EL TOBILLO


## SERVICIO

Ahora para poder determinar las especialidades, usamos el dataset de PASTO, que ya fue analizado.

In [32]:
# URL directa al archivo en formato .xlsx en GitHub
url_excel = 'https://raw.githubusercontent.com/aisaza609/Datos_a_la_U_grupo_2/main/Dataset%20de%20datos%20abiertos/Hospital_Pasto_v1.xlsx'

# Descargar y cargar el archivo .xlsx
response_excel = requests.get(url_excel)
if response_excel.status_code == 200:
    df_pasto = pd.read_excel(BytesIO(response_excel.content), engine='openpyxl')
    # Mostrar las primeras filas para verificar

else:
    print("No se pudo descargar el archivo .xlsx.")

Se hace hace el procesamiento de las impresiones_diagnositcas, para que queden las dos columnas. Codigo y Nombre Diagnostico

In [33]:
# Usamos str.extract() con una expresión regular para extraer el código y el nombre del diagnóstico
df_pasto[['CodigoDiagnostico', 'Nombre Diagnostico']] = df_pasto['Impresiones_Diagnosticas'].str.extract(r'Diagnóstico Pincipal: (\w+) - (.+)')

# Mostramos las primeras filas para verificar
print(df_pasto[['Impresiones_Diagnosticas', 'CodigoDiagnostico', 'Nombre Diagnostico']].head())

                            Impresiones_Diagnosticas CodigoDiagnostico  \
0  Diagnóstico Pincipal: K069 - TRASTORNO NO ESPE...              K069   
1  Diagnóstico Pincipal: K047 - ABSCESO PERIAPICA...              K047   
2  Diagnóstico Pincipal: I443 - OTROS TIPOS DE BL...              I443   
3  Diagnóstico Pincipal: E45X - RETARDO DEL DESAR...              E45X   
4  Diagnóstico Pincipal: M751 - SINDROME DE MANGUITO              M751   

      Nombre Diagnostico  
0  TRASTORNO NO ESPECIFI  
1  ABSCESO PERIAPICAL SI  
2  OTROS TIPOS DE BLOQUE  
3  RETARDO DEL DESARROLL  
4   SINDROME DE MANGUITO  


Ahora dependiendo del codigo, le hacemos la asignacion de servicios con la informacion ya implementada en el dataset de pasto, ya que se hizo previamente en el analisis descriptivo

In [34]:
# Verifico que la columna 'CodigoDiagnostico' y 'Objeto_Remision' existan en df_pasto después de la extracción
print("Columnas en df_pasto:", df_pasto.columns)

# Si 'CodigoDiagnostico' o 'Objeto_Remision' no están en la lista, reviso la extracción
if 'CodigoDiagnostico' not in df_pasto.columns or 'Objeto_Remision' not in df_pasto.columns:
    print("La columna 'CodigoDiagnostico' o 'Objeto_Remision' no existe en df_pasto. Asegúrate de que la extracción fue exitosa.")
else:
    # Realizo un merge entre df_neiva y df_pasto usando la columna 'CodigoDiagnostico' en ambos DataFrames
    df_neiva = df_neiva.merge(df_pasto[['CodigoDiagnostico', 'Objeto_Remision']], on='CodigoDiagnostico', how='left')

    # Renombro la columna 'Objeto_Remision' en df_neiva como 'Servicio'
    df_neiva = df_neiva.rename(columns={'Objeto_Remision': 'Servicio'})

    # Verifico el resultado
    print(df_neiva[['CodigoDiagnostico', 'Servicio']].head())


Columnas en df_pasto: Index(['IPS', 'FechaAsignacionRegistro', 'Sexo', 'Edad', 'Hora_Salida',
       'Objeto_Remision', 'Impresiones_Diagnosticas', 'NivelHospital',
       'NiveldeAtencionNecesario', 'Traslado', 'RangoEdad',
       'CodigoDiagnostico', 'Nombre Diagnostico'],
      dtype='object')
  CodigoDiagnostico         Servicio
0              D172  CIRUGIA GENERAL
1              D172  CIRUGIA GENERAL
2              D172  CIRUGIA GENERAL
3              D172  CIRUGIA GENERAL
4              D172     DERMATOLOGIA


Con el analisis de unos codigos especificos de medico general se asigna servicios especifios.

In [35]:
# Completar manualmente los valores en 'Servicio' en df_neiva según los códigos específicos
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['R509', 'S019', 'A099', 'R31X', 'I500', 'J46X', 'K297', 'I872']), 'Servicio'] = 'MEDICINA INTERNA'
df_neiva.loc[df_neiva['CodigoDiagnostico'] == 'R074', 'Servicio'] = 'MEDICINA INTERNA'
df_neiva.loc[df_neiva['CodigoDiagnostico'] == 'N23X', 'Servicio'] = 'MEDICINA INTERNA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['J039', 'R529', 'G439', 'T172', 'H669', 'I839']), 'Servicio'] = 'MEDICINA GENERAL'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['R456']), 'Servicio'] = 'PSICOLOGIA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['G409', 'S098']), 'Servicio'] = 'NEUROLOGIA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['S810', 'V99X', 'S501', 'S801']), 'Servicio'] = 'ORTOPEDIA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['J459']), 'Servicio'] = 'NEUMOLOGIA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['M549']), 'Servicio'] = 'FISIATRIA'
df_neiva.loc[df_neiva['CodigoDiagnostico'].isin(['R104']), 'Servicio'] = 'CIRUGIA GENERAL'

# Verificar el resultado
print(df_neiva[['CodigoDiagnostico', 'Servicio']].head())


  CodigoDiagnostico         Servicio
0              D172  CIRUGIA GENERAL
1              D172  CIRUGIA GENERAL
2              D172  CIRUGIA GENERAL
3              D172  CIRUGIA GENERAL
4              D172     DERMATOLOGIA


## Nivel de atencion y Traslado

Mapeamos los servicios segun la especialidad a las que deben dirigirse los pacientes: El diccionario de especialidades médicas se clasifica en tres niveles de atención:

Nivel 1: Especialidades básicas de atención primaria, como medicina general y urgencias basicas, que no requieren tecnología avanzada.
Nivel 2: Especialidades de complejidad intermedia, como Mediciona Interna y ortopedia, que pueden necesitar algunos equipos especializados.
Nivel 3: Especialidades de alta complejidad, como neurocirugía y oncología, que requieren instalaciones avanzadas y equipos altamente especializados.

In [36]:
# Definir el mapeo de especialidades a niveles de atención con términos clave
nivel_atencion_map = {
    "nivel 1": ["medicina general", "nutricion", "psicologia", "terapia fisica", "rehabilitacion","optometria"],
    "nivel 2": ["medicina interna", "cirugia general", "ortopedia", "ginecologia", "pediatria", "urologia", "psiquiatria", "alergologia","pomeroy","vasectomia"],
    "nivel 3": ["cardiologia", "cirugia plastica", "cirugia vascular", "dermatologia", "endocrinologia", "fisiatria",
                "gastroenterologia", "genetica", "hematologia", "nefrologia", "neumologia", "neurocirugia",
                "neurologia", "neuropediatria", "oftalmologia", "oncologia", "otorrinolaringologia", "reumatologia"]
}

# Función para asignar el nivel de atención según el objeto de remisión
def asignar_nivel_atencion(Servicio):
    Servicio_lower = str(Servicio).lower()  # Convertir a minúsculas para comparación
    for nivel, especialidades in nivel_atencion_map.items():
        # Verificamos si alguna palabra clave de la especialidad está en el objeto de remisión
        if any(especialidad in Servicio_lower for especialidad in especialidades):
            return int(nivel.split()[-1])  # Devolver solo el número de nivel
    return None  # Si no se encuentra, devolver None

# Aplicar la función para crear la columna 'NiveldeAtencionNecesario'
df_neiva['NiveldeAtencionNecesario'] = df_neiva['Servicio'].apply(asignar_nivel_atencion)



# Mostrar el conteo de niveles asignados para verificar
print(df_neiva['NiveldeAtencionNecesario'].value_counts())


NiveldeAtencionNecesario
2.0    989425
1.0    426017
3.0    390364
Name: count, dtype: int64


Considerando que este hospital es un nivel 3, tiene los servicios adecuados para mantener a sus pacientes por sus servicios, aun no se tienen en cuenta los que deben ser traslados por su EPS.

In [37]:
# Definir el nivel del hospital actual
nivel_hospital = 3

# Crear la función 'definir_traslado' simplificada para el hospital de nivel 3
def definir_traslado(nivel_necesario):
    return "No necesita traslado"  # Siempre devuelve que no se necesita traslado

# Aplicar la función para crear la columna 'Traslado'
df_neiva['Traslado'] = df_neiva['NiveldeAtencionNecesario'].apply(definir_traslado)

# Verificar el resultado
print(df_neiva[['NiveldeAtencionNecesario', 'Traslado']].head())


   NiveldeAtencionNecesario              Traslado
0                       2.0  No necesita traslado
1                       2.0  No necesita traslado
2                       2.0  No necesita traslado
3                       2.0  No necesita traslado
4                       3.0  No necesita traslado


Para que todas las columnas queden en el mismo orden y mismo nombre que los demas datasets

In [38]:
orden_columnas = [
    'tipo de identificación',
    'NDocumento',
    'Sexo',
    'Edad',
    'Ingreso',
    'FechaIngreso',
    'Entidad',
    'CodigoDiagnostico',
    'NombreDiagnostico',
    'DxSindromatico',
    'NiveldeAtencionNecesario',
    'Servicio',
    'NivelHospital',
    'Traslado'
]
# Reordenar el DataFrame df_neiva con el nuevo orden de columnas
df_neiva = df_neiva[orden_columnas]
df_neiva

,tipo de identificación,NDocumento,Sexo,Edad,Ingreso,FechaIngreso,Entidad,CodigoDiagnostico,NombreDiagnostico,DxSindromatico,NiveldeAtencionNecesario,Servicio,NivelHospital,Traslado
0,CC,6362268471,F,52,5665894,2023-12-04,PARTICULARES,D172,TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...,TUMORES BENIGNOS LIPOMATOSOS,2.0,CIRUGIA GENERAL,3,No necesita traslado
1,CC,6362268471,F,52,5665894,2023-12-04,PARTICULARES,D172,TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...,TUMORES BENIGNOS LIPOMATOSOS,2.0,CIRUGIA GENERAL,3,No necesita traslado
2,CC,6362268471,F,52,5665894,2023-12-04,PARTICULARES,D172,TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...,TUMORES BENIGNOS LIPOMATOSOS,2.0,CIRUGIA GENERAL,3,No necesita traslado
3,CC,6362268471,F,52,5665894,2023-12-04,PARTICULARES,D172,TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...,TUMORES BENIGNOS LIPOMATOSOS,2.0,CIRUGIA GENERAL,3,No necesita traslado
4,CC,6362268471,F,52,5665894,2023-12-04,PARTICULARES,D172,TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...,TUMORES BENIGNOS LIPOMATOSOS,3.0,DERMATOLOGIA,3,No necesita traslado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119247,CC,5561935546,M,64,6408391,2022-03-31,ALIANZA MEDELLIN ANTIOQUIA SAS,J441,ENFERMEDAD PULMONAR OBSTRUCTIVA CRONICA CON EX...,OTRAS ENFERMEDADES PULMONARES OBSTRUCTIVAS CRO...,2.0,MEDICINA INTERNA,3,No necesita traslado
2119248,CC,5561935546,M,64,6408391,2022-03-31,ALIANZA MEDELLIN ANTIOQUIA SAS,J441,ENFERMEDAD PULMONAR OBSTRUCTIVA CRONICA CON EX...,OTRAS ENFERMEDADES PULMONARES OBSTRUCTIVAS CRO...,2.0,MEDICINA INTERNA,3,No necesita traslado
2119249,CC,5561935546,M,64,6408391,2022-03-31,ALIANZA MEDELLIN ANTIOQUIA SAS,J441,ENFERMEDAD PULMONAR OBSTRUCTIVA CRONICA CON EX...,OTRAS ENFERMEDADES PULMONARES OBSTRUCTIVAS CRO...,2.0,MEDICINA INTERNA,3,No necesita traslado
2119250,CC,5561935546,M,64,6408391,2022-03-31,ALIANZA MEDELLIN ANTIOQUIA SAS,J441,ENFERMEDAD PULMONAR OBSTRUCTIVA CRONICA CON EX...,OTRAS ENFERMEDADES PULMONARES OBSTRUCTIVAS CRO...,2.0,MEDICINA INTERNA,3,No necesita traslado
